In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,13719
2,Huelva,Confirmados PDIA 14 días,2013
3,Huelva,Tasa PDIA 14 días,"394,1316865820971"
4,Huelva,Confirmados PDIA 7 días,1474
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,59
629,Municipio de Huelva sin especificar,Total Confirmados,217
630,Municipio de Huelva sin especificar,Curados,114


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  13719.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  3882.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 360 personas en los últimos 7 días 

Un positivo PCR cada 262 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,13719.0,1474.0,2013.0,510743.0,288.599158,394.131687,404.0
Huelva-Costa,7933.0,918.0,1284.0,288115.0,318.622772,445.655381,272.0
Huelva (capital),3882.0,399.0,549.0,143663.0,277.733306,382.144324,116.0
Condado-Campiña,3999.0,346.0,442.0,155057.0,223.143747,285.056463,88.0
Isla Cristina,820.0,223.0,309.0,21264.0,1048.720843,1453.160271,81.0
Sierra de Huelva-Andévalo Central,1571.0,151.0,215.0,67571.0,223.468648,318.183836,28.0
Villalba del Alcor,206.0,92.0,101.0,3338.0,2756.141402,3025.763930,21.0
Lepe,1103.0,74.0,103.0,27431.0,269.767781,375.487587,18.0
Palma del Condado (La),649.0,65.0,77.0,10761.0,604.033082,715.546882,18.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Encinasola,59.0,46.0,49.0,1305.0,3524.904215,3754.789272,9.0
Villalba del Alcor,206.0,92.0,101.0,3338.0,2756.141402,3025.763930,21.0
Villanueva de las Cruces,7.0,7.0,7.0,373.0,1876.675603,1876.675603,5.0
Castaño del Robledo,6.0,3.0,4.0,223.0,1345.291480,1793.721973,0.0
Isla Cristina,820.0,223.0,309.0,21264.0,1048.720843,1453.160271,81.0
Calañas,68.0,18.0,23.0,2791.0,644.930133,824.077392,4.0
Cerro de Andévalo (El),70.0,9.0,17.0,2364.0,380.710660,719.120135,2.0
Palma del Condado (La),649.0,65.0,77.0,10761.0,604.033082,715.546882,18.0
Aroche,75.0,7.0,20.0,3073.0,227.790433,650.829808,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Zarza-Perrunal (La),16.0,1.0,3.0,1267.0,78.926598,236.779795,0.0,0.333333
Corteconcepción,21.0,1.0,3.0,548.0,182.481752,547.445255,0.0,0.333333
Aroche,75.0,7.0,20.0,3073.0,227.790433,650.829808,1.0,0.350000
San Bartolomé de la Torre,58.0,6.0,13.0,3736.0,160.599572,347.965739,3.0,0.461538
Almonaster la Real,37.0,2.0,4.0,1819.0,109.950522,219.901045,0.0,0.500000
Gibraleón,360.0,1.0,2.0,12607.0,7.932101,15.864202,0.0,0.500000
Campillo (El),57.0,1.0,2.0,2023.0,49.431537,98.863075,0.0,0.500000
Cerro de Andévalo (El),70.0,9.0,17.0,2364.0,380.710660,719.120135,2.0,0.529412
Aracena,333.0,15.0,26.0,8107.0,185.025287,320.710497,1.0,0.576923
